In [87]:
import pandas as pd
import numpy as np
import collections
import ast


In [121]:
df = pd.read_csv("analysed cc - first pass.csv")
#contains the dictionary where keys are specific key words, and values are a tuple of (x_bar, n)
key_words = {
    'lecture' : [0,0],
    'material' : [0,0],
    'information' : [0,0], 
    'organization' : [0,0],
    'disorganized' : [0,0],
    'concepts' : [0,0],
    'discussion' : [0,0],
    'memorization' : [0,0],
    'workload' : [0,0],
    'read' : [0,0],
    'exams' : [0,0],
    'presentation' : [0,0],
    'homework' : [0,0],
    'labs' : [0,0],
    'textbook' : [0,0],
    'curve' : [0,0],
    'recitation' : [0,0]
} 

#df['tweet'].info()

In [122]:
# function that goes in lambda expression below
def update_weight(input): # input is a tuple where input[0] is list of words and input[1] is rating
    for x in ast.literal_eval(input[0]):
        #print(x)
        if x in key_words: # key_words dictionary
            key_words[x][0] = (key_words[x][0] * key_words[x][1] + int(input[1])) / (key_words[x][1] + 1)
            key_words[x][1] += 1

In [123]:
for index, row in df[["tweet", "classified"]].iterrows():
    update_weight((row['tweet'], row['classified']))

In [124]:
print(key_words)

{'lecture': [3.8449388209121245, 4495], 'material': [3.969456427955142, 5795], 'information': [4.014520813165534, 1033], 'organization': [3.789156626506024, 332], 'disorganized': [0, 0], 'concepts': [3.935374149659867, 1470], 'discussion': [4.060606060606061, 726], 'memorization': [4.078059071729952, 474], 'workload': [4.114638447971787, 567], 'read': [4.026041666666659, 2688], 'exams': [3.702045542261674, 2591], 'presentation': [3.8220338983050848, 118], 'homework': [3.8418700584393264, 2909], 'labs': [3.601983002832861, 706], 'textbook': [3.7438775510204083, 980], 'curve': [3.2548262548262556, 518], 'recitation': [3.7142857142857144, 35]}


In [126]:
for x in key_words.keys():
    if key_words[x] == [0,0]:
        del key_words[x]

In [127]:
print(key_words)

{'lecture': [3.8449388209121245, 4495], 'material': [3.969456427955142, 5795], 'information': [4.014520813165534, 1033], 'organization': [3.789156626506024, 332], 'concepts': [3.935374149659867, 1470], 'discussion': [4.060606060606061, 726], 'memorization': [4.078059071729952, 474], 'workload': [4.114638447971787, 567], 'read': [4.026041666666659, 2688], 'exams': [3.702045542261674, 2591], 'presentation': [3.8220338983050848, 118], 'homework': [3.8418700584393264, 2909], 'labs': [3.601983002832861, 706], 'textbook': [3.7438775510204083, 980], 'curve': [3.2548262548262556, 518], 'recitation': [3.7142857142857144, 35]}
